In [ ]:
from konlpy.tag import Okt
from gensim.models import word2vec
import codecs

f = open("c:/data/text/ratings.csv", encoding="ms949")
data = f.read()
twitter = Okt()
lines = data.split("\n")
results = []
for line in lines[:1000]:
    r = []
    word_list = twitter.pos(line, norm=True, stem=True)
    for (word, pumsa) in word_list:
        if not pumsa in ["Josa", "Eomi", "Punctuation"]:
            r.append(word)
    results.append((" ".join(r)).strip())

output = (" ".join(results)).strip()
with open("c:/data/text/text_prepared.dat", "w", encoding="utf-8") as fp:
    fp.write(output)

print("작업완료")

In [ ]:
data = word2vec.LineSentence("c:/data/text/text_prepared.dat")
model = word2vec.Word2Vec(data, vector_size=100, window=10, hs=1, min_count=2, sg=1)
model.save("c:/data/text/text_100.model")
print("word2vec 모델 생성 완료")

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

text = open("c:/data/text/ratings.csv", encoding="ms949").read()
wordcloud = WordCloud(font_path = "c:/Windows/Fonts/malgun.ttf").generate(text)
wordcloud.words_

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
model = word2vec.Word2Vec.load("c:/data/text/text_100.model")
print(model.wv.most_similar(positive=["영화"]))

In [1]:
import numpy as np
import csv

title_arr = []
ctr_arr = []
f = open("c:/data/text/ratings_train.csv", encoding="ms949")
reader = csv.reader(f)
max_length = 0
length = 0
max_idx = 0
count = 0
next(reader, None)
count_train = 0
for line in reader:
    title_arr.append(line[0])
    ctr_arr.append(int(line[1]))
    length = len(line[0])
    if max_length < length:
        max_length = length
        max_idx = count
    count += 1
    count_train += 1
    if count_train >= 10000: break
f.close()

f2 = open("c:/data/text/ratings_test.csv", encoding="ms949")
reader = csv.reader(f2)
next(reader, None)
count_test = 0
for line in reader:
    title_arr.append(line[0])
    ctr_arr.append(int(line[1]))
    length = len(line[0])
    if max_length < length:
        max_length = length
        max_idx = count
    count += 1
    count_test += 1
    if count_test >= 10000: break
f2.close()

print("count_train : ",count_train)
print("count_test : ",count_test)
print("title_arr size : ",len(title_arr))
print("ctr_arr size : ",len(ctr_arr))
print("max length : ",max_length)

count_train :  10000
count_test :  10000
title_arr size :  20000
ctr_arr size :  20000
max length :  144


In [2]:
from konlpy.tag import Okt

okt = Okt()
title_noun_arr = []
for index, title in enumerate(title_arr):
    if index % 1000 == 0:
        print("step : ",index)
    title_noun_arr.append(okt.nouns(title))
print(len(title_noun_arr))

step :  0
step :  1000
step :  2000
step :  3000
step :  4000
step :  5000
step :  6000
step :  7000
step :  8000
step :  9000
step :  10000
step :  11000
step :  12000
step :  13000
step :  14000
step :  15000
step :  16000
step :  17000
step :  18000
step :  19000
20000


In [3]:
import gensim

MIN_NOUN_VECTOR_VALUE = -10.0
MAX_NOUN_VECTOR_VALUE = 10.0
NOUN_VECTOR_SIZE = 100 # 벡터 사이즈 100차원. 위에 만든 모델이 100차원이었으니까 맞춰서

def generate_random_noun_vector():
    return np.random.uniform(low=MIN_NOUN_VECTOR_VALUE, high=MAX_NOUN_VECTOR_VALUE, size=(NOUN_VECTOR_SIZE,))

w2v_model = gensim.models.Word2Vec.load("c:/data/text/text_100.model")
title_noun_vector_arr = []
for index, title_nouns in enumerate(title_noun_arr):
    if index % 10000 == 0:
        print("step : ", index)
    noun_vector_arr = []
    for noun in title_nouns:
        try:
            noun_vector = w2v_model[noun] # word2vec에 단어 입력해서 100차원으로 변환
        except Exception as e: # 단어 사전에 없으면 랜덤으로 처리
            noun_vector = generate_random_noun_vector()
        noun_vector_arr.append(noun_vector)
    title_noun_vector_arr.append(noun_vector_arr)

print(title_noun_vector_arr[:1])

step :  0
step :  10000
[[array([ 2.9211516 ,  2.07885795,  9.6879388 , -2.06501218,  3.18258903,
        3.8852878 , -5.35468854, -4.25787805, -5.54966304, -7.45813175,
        0.38150759,  9.82161352,  8.11490294, -0.14043681,  6.92923098,
        4.74238594,  9.50121344,  9.55777134,  7.42406869, -5.79672672,
       -4.66598523, -5.34374644, -6.63820544,  5.32099849,  9.93356046,
       -6.4968982 ,  6.14494617, -3.02361303, -9.25553307, -3.12678824,
        6.29089512, -2.41580013,  7.88240341,  3.83309147,  7.08591694,
       -5.76136884,  7.35382002, -1.10585173, -2.62531771,  8.50424913,
        7.00828959, -7.82105791, -8.188705  ,  7.18123446,  3.98857505,
        9.11610603, -2.221377  , -6.70806401,  5.76341386,  4.44447788,
        7.08854003, -8.45052209,  6.32465538,  2.7662275 ,  0.5537454 ,
       -4.40755091,  2.00725984, -1.83469736,  3.28588288,  4.86423799,
       -9.39979609, -6.63301735, -8.87126506,  3.08256869, -2.72688893,
        2.04318751, -4.58063358, -5.11

In [4]:
max_size = 0
min_size = 100
avg = 0
count = 0
sum_size = 0

for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if len(title_noun_vector) == 0:
        pass
    sum_size += len(title_noun_vector)
    if max_size < len(title_noun_vector):
        max_size = len(title_noun_vector)
    if min_size > len(title_noun_vector):
        min_size = len(title_noun_vector)
    count += 1

TITLE_LENGTH = max_size

def generate_zero_noun_vector(): # 제로패딩 준비
    return np.random.uniform(low=0.0, high=0.0, size=(NOUN_VECTOR_SIZE,))

title_noun_vector_arr2 = [] # 최대사이즈에 맞게 제로패딩
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if index % 1000 == 0:
        print('step : ',index)
    while len(title_noun_vector) < TITLE_LENGTH:
        title_noun_vector.append(generate_zero_noun_vector())
    title_noun_vector = title_noun_vector[:TITLE_LENGTH]
    title_noun_vector_arr2.append(title_noun_vector)

print(min_size, max_size)

step :  0
step :  1000
step :  2000
step :  3000
step :  4000
step :  5000
step :  6000
step :  7000
step :  8000
step :  9000
step :  10000
step :  11000
step :  12000
step :  13000
step :  14000
step :  15000
step :  16000
step :  17000
step :  18000
step :  19000
0 37


In [5]:
NUM_CLASSES = 2
ctr_class_arr = []
ctr_class_count = [0,0]

for index, ctr in enumerate(ctr_arr):
    if ctr == 0: # 부정리뷰
        ctr_class_arr.append(0.0)
        ctr_class_count[0] += 1
    elif ctr == 1: # 긍정리뷰
        ctr_class_arr.append(1.0)
        ctr_class_count[1] += 1
    
print(ctr_class_count)
print(ctr_class_arr[:10])

[9960, 10040]
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]


In [6]:
def prepare_data(arr):
    sum1 = 0
    for i in arr:
        sum1 += i
    arr = [float(i)/sum1 for i in arr]
    return arr

In [7]:
# 학습용 검증용 나누기

test_data_size = count_test
train_data_size = count_train

train_input = title_noun_vector_arr[0:train_data_size]
train_label = ctr_class_arr[0:train_data_size]
test_input = title_noun_vector_arr[train_data_size:]
test_label = ctr_class_arr[train_data_size:]

print("ctr_class_arr 개수 : ",len(ctr_class_arr))
print("train 데이터 개수 : ",len(train_input))
print("train 라벨 개수 : ",len(train_label))
print("test 데이터 개수 : ",len(test_input))
print("test 라벨 개수 : ",len(test_label))
print("train_data_size : ",train_data_size)
print("test_data_size : ",test_data_size)

ctr_class_arr 개수 :  20000
train 데이터 개수 :  10000
train 라벨 개수 :  10000
test 데이터 개수 :  10000
test 라벨 개수 :  10000
train_data_size :  10000
test_data_size :  10000


In [8]:
# keras에 넣기 위해 차원 변경

train_input = np.array(train_input)
train_input = train_input.reshape(train_input.shape[0], train_input.shape[1], NOUN_VECTOR_SIZE, 1)
test_input = np.array(test_input)
test_input = test_input.reshape(test_input.shape[0], test_input.shape[1], NOUN_VECTOR_SIZE, 1)

In [9]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

input_a = Input(shape=(train_input.shape[1], NOUN_VECTOR_SIZE, 1), name="input-layer")
x = Conv2D(1, (3,3), activation='relu', padding='valid', strides=(1,1))(input_a) # 3*3짜리 필터 한 개, 패딩을 통해 인풋과 아웃풋의 사이즈 똑같게, stride는 쉬프트 값
x = Conv2D(1, (3,3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x) # 1차원으로 변환
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid', name='output-layer')(x)
model = Model(inputs=[input_a], outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=2)
hist = model.fit(x=train_input, y=np.array(train_label), validation_split=0.25, batch_size=256, epochs=5, verbose=1, callbacks=[early_stopping])

Epoch 1/5
30/30 [==============================] - 6s 199ms/step - loss: 1.1860 - accuracy: 0.5114 - val_loss: 0.7008 - val_accuracy: 0.5228
Epoch 2/5
30/30 [==============================] - 5s 151ms/step - loss: 0.7277 - accuracy: 0.4977 - val_loss: 0.6940 - val_accuracy: 0.5112
Epoch 3/5
30/30 [==============================] - 5s 152ms/step - loss: 0.7020 - accuracy: 0.5057 - val_loss: 0.6935 - val_accuracy: 0.5132
Epoch 4/5
30/30 [==============================] - 5s 153ms/step - loss: 0.6945 - accuracy: 0.5164 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 5/5
30/30 [==============================] - 5s 152ms/step - loss: 0.6951 - accuracy: 0.5108 - val_loss: 0.6935 - val_accuracy: 0.5096


In [11]:
scores = model.evaluate(np.array(test_input)[:100], np.array(test_label)[:100], batch_size=1, verbose=1)
print("%s: %.2f%%"%(model.metrics_names[0], scores[0]*100))
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

m2 = Model(inputs=model.input, outputs=model.get_layer('output-layer').output)
y = m2.predict(test_input)

100/100 [==============================] - 0s 2ms/step - loss: 0.6890 - accuracy: 0.5500
loss: 68.90%
accuracy: 55.00%


In [12]:
y[0:10]

array([[0.500761  ],
       [0.500761  ],
       [0.47888556],
       [0.5047337 ],
       [0.50270516],
       [0.52642244],
       [0.48652118],
       [0.49697497],
       [0.48269248],
       [0.49538818]], dtype=float32)

In [13]:
model.save("c:/data/text")

INFO:tensorflow:Assets written to: c:/data/text\assets


In [14]:
n = 911
print(model.predict(np.array(train_input)[n].reshape(1,37,100,1)))
print(np.array(train_label)[n])
print(title_arr[n])

[[0.500761]]
1.0
재밌군


In [ ]:
# rnn = 시퀀스. 문장의 배열, 단어 순서까지 고려. 얘가 쪼꼼 더 쉬움 ㅎㅎ
# cnn = word2vec이 단어 순서를 고려